In [2]:
using DifferentialEquations, DiffEqProblemLibrary, ParameterizedFunctions, Plots, ODE, ODEInterfaceDiffEq
gr()

van = @ode_def_noinvhes VanDerPol2 begin
  dy = μ*((1-x^2)*y - x)
  dx = 1*y
end μ=>1.

prob = ODEProblem(VanDerPol2(μ=1e6),[0;2.],(0.0,6.3))
abstols = 1./10.^(5:9)
reltols = 1./10.^(2:6)

sol = solve(prob,radau(),abstol=1/10^14,reltol=1/10^14)
test_sol = TestSolution(sol)
sol2 = solve(prob,Rodas4(),abstol=1/10^11,reltol=1/10^11,maxiters=1e7)
test_sol2 = TestSolution(sol2)

retcode: Success
Interpolation: specialized 3rd order "free" stiffness-aware interpolation
t: nothing
u: nothing

### Plot Test

In [5]:
plot(sol,ylim=[-4;4])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,184.252 39.3701,184.34 39.3701,184.428 39.3701,184.745 39.3701,185.059 39.3701,185.369 39.3701,185.676 39.3701,185.966 39.3701,186.253 39.3701,186.536 
 39.3701,186.817 39.3701,188.96 39.3701,193.303 39.3701,196.867 39.3701,199.806 39.3701,202.23 39.3701,204.23 39.3701,205.88 39.3701,207.516 39.3701,208.808 
 39.3702,211.705 39.3702,212.868 39.3702,213.394 39.3703,213.565 39.3703,213.629 39.3703,213.644 39.3704,213.648 39.3705,213.648 39.3706,213.648 39.3707,213.648 
 39.371,213.648 39.3716,213.649 39.3749,213.649 39.3965,213.653 39.5064,213.674 40.3854,213.839 44.7723,214.713 50.2409,215.93 55.7095,217.321 61.66,219.085 
 66.8514,220.905 71.4764,222.818 75.602,224.831 79.2815,226.949 82.5628,229.178 85.4887,231.522 88.0977,233.987 90.4241,236.578 92.4989,239.302 94.3496,242.165 
 96.0007,245.173 97.4742,248.333 98.7895,251.652 99.9641,255.138 101.013,258.799 101.951,262.643 102.789,266.679 103.539,270.915 104.21,275.361 104.811,280.028 
 105.349,284.924 105.831,290.06 106.263,295.449 106.651,301.101 106.998,307.027 107.32,313.458 107.6,320.003 107.85,326.835 108.083,334.246 108.285,341.789 
 108.472,349.92 108.64,358.49 108.786,367.161 108.921,376.488 109.043,386.323 109.149,396.26 109.246,406.911 109.335,418.146 109.412,429.478 109.483,441.576 
 109.547,454.34 109.605,467.646 109.657,481.487 109.704,495.899 109.746,510.914 109.784,526.555 109.819,542.844 109.85,559.804 109.878,577.464 109.903,595.848 
 109.926,614.981 109.947,634.879 109.967,656.294 109.985,678.724 110.001,701.948 110.015,725.966 110.028,750.777 110.04,776.355 110.051,802.673 110.06,829.711 
 110.069,857.468 110.077,885.954 110.084,916.274 110.091,947.637 110.097,979.945 110.102,1013.24 110.108,1047.59 110.112,1083.08 110.117,1119.78 110.12,1157.8 
 110.124,1198.72 110.128,1241.43 110.131,1285.9 110.134,1332.25 110.137,1380.62 110.139,1431.16 110.142,1484.03 110.144,1539.4 110.146,1597.45 110.148,1658.38 
 110.15,1722.42 110.152,1789.78 110.153,1860.73 110.155,1935.52 110.156,2014.47 110.157,2097.88 110.159,2186.09 110.16,2283.16 110.161,2382.51 110.162,2491.89 
 110.163,2608.47 110.164,2732.66 110.165,2865.11 110.166,3006.49 110.168,3534.05 110.17,4248.03 110.172,5093.61 110.173,6165.63 110.175,7550.64 110.176,9338.29 
 110.177,11653 110.178,14661.8 110.178,18583.3 110.179,23705.5 110.179,30407.6 110.18,39189.7 110.18,50711.7 110.181,65844.6 110.181,85740.3 110.181,111924 
 110.181,146418 110.182,191911 110.182,251987 110.182,331436 110.182,436689 110.182,576426 110.182,762438 110.182,1.01088e+06 110.182,1.34418e+06 110.183,1.79396e+06 
 110.183,2.40604e+06 110.183,3.24957e+06 110.183,4.43589e+06 110.183,6.14167e+06 110.183,8.79336e+06 110.183,1.15452e+07 110.183,1.49648e+07 110.183,1.97303e+07 110.183,2.59295e+07 110.183,3.42695e+07 
 110.183,4.16655e+07 110.183,4.86854e+07 110.183,5.55073e+07 110.183,5.87103e+07 110.183,5.75381e+07 110.183,5.13505e+07 110.183,4.36588e+07 110.183,3.43461e+07 110.183,2.66682e+07 110.183,1.82412e+07 
 110.183,1.16941e+07 110.183,7.51936e+06 110.183,4.38752e+06 110.183,2.41051e+06 110.183,1.36736e+06 110.183,772033 110.183,406617 110.183,186041 110.183,85063.1 110.183,42675.1 
 110.183,21731.6 110.183,11192 110.183,5822.22 110.183,3070.25 110.183,1655.84 110.183,927.627 110.183,552.711 110.183,359.689 110.183,260.313 110.183,209.15 
 110.183,182.809 110.183,169.248 110.184,162.266 110.184,158.073 110.184,156.253 110.184,155.349 110.184,155.031 110.184,154.903 110.184,154.869 110.184,154.859 
 110.184,154.858 110.184,154.857 110.184,154.857 110.184,154.857 110.185,154.857 110.186,154.857 110.192,154.856 110.22,154.851 110.385,154.82 111.702,154.57 
 117.717,153.337 133.169,149.195 139.935,146.703 146.702,143.525 15

In [6]:
plot(sol)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 -1000000 
 
 
 0 
 
 
 1000000 
 
 
 t 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 
 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 39.3701,184.254 
 39.3702,184.254 39.3702,184.254 39.3702,184.254 39.3703,184.254 39.3703,184.254 39.3703,184.254 39.3704,184.254 39.3705,184.254 39.3706,184.254 39.3707,184.254 
 39.371,184.254 39.3716,184.254 39.3749,184.254 39.3965,184.254 39.5064,184.254 40.3854,184.254 44.7723,184.254 50.2409,184.254 55.7095,184.254 61.66,184.254 
 66.8514,184.254 71.4764,184.254 75.602,184.254 79.2815,184.254 82.5628,184.254 85.4887,184.254 88.0977,184.254 90.4241,184.254 92.4989,184.254 94.3496,184.254 
 96.0007,184.254 97.4742,184.254 98.7895,184.254 99.9641,184.254 101.013,184.254 101.951,184.254 102.789,184.254 103.539,184.254 104.21,184.254 104.811,184.254 
 105.349,184.254 105.831,184.254 106.263,184.254 106.651,184.254 106.998,184.254 107.32,184.254 107.6,184.254 107.85,184.255 108.083,184.255 108.285,184.255 
 108.472,184.255 108.64,184.255 108.786,184.255 108.921,184.255 109.043,184.255 109.149,184.255 109.246,184.255 109.335,184.255 109.412,184.255 109.483,184.255 
 109.547,184.255 109.605,184.255 109.657,184.255 109.704,184.255 109.746,184.255 109.784,184.255 109.819,184.255 109.85,184.255 109.878,184.255 109.903,184.255 
 109.926,184.255 109.947,184.255 109.967,184.256 109.985,184.256 110.001,184.256 110.015,184.256 110.028,184.256 110.04,184.256 110.051,184.256 110.06,184.256 
 110.069,184.256 110.077,184.256 110.084,184.256 110.091,184.256 110.097,184.256 110.102,184.257 110.108,184.257 110.112,184.257 110.117,184.257 110.12,184.257 
 110.124,184.257 110.128,184.257 110.131,184.257 110.134,184.258 110.137,184.258 110.139,184.258 110.142,184.258 110.144,184.258 110.146,184.258 110.148,184.259 
 110.15,184.259 110.152,184.259 110.153,184.259 110.155,184.259 110.156,184.26 110.157,184.26 110.159,184.26 110.16,184.26 110.161,184.261 110.162,184.261 
 110.163,184.261 110.164,184.262 110.165,184.262 110.166,184.263 110.168,184.264 110.17,184.266 110.172,184.269 110.173,184.272 110.175,184.276 110.176,184.282 
 110.177,184.288 110.178,184.298 110.178,184.309 110.179,184.325 110.179,184.345 110.18,184.371 110.18,184.406 110.181,184.451 110.181,184.511 110.181,184.589 
 110.181,184.693 110.182,184.829 110.182,185.009 110.182,185.248 110.182,185.563 110.182,185.983 110.182,186.54 110.182,187.286 110.182,188.285 110.183,189.635 
 110.183,191.471 110.183,194.001 110.183,197.559 110.183,202.676 110.183,210.629 110.183,218.884 110.183,229.141 110.183,243.435 110.183,262.03 110.183,287.046 
 110.183,309.23 110.183,330.286 110.183,350.749 110.183,360.356 110.183,356.84 110.183,338.281 110.183,315.209 110.183,287.276 110.183,264.245 110.183,238.968 
 110.183,219.33 110.183,206.808 110.183,197.414 110.183,191.484 110.183,188.355 110.183,186.569 110.183,185.473 110.183,184.812 110.183,184.509 110.183,184.382 
 110.183,184.319 110.183,184.287 110.183,184.271 110.183,184.263 110.183,184.259 110.183,184.256 110.183,184.255 110.183,184.255 110.183,184.254 110.183,184.254 
 110.183,184.254 110.183,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 
 110.184,184.254 110.184,184.254 110.184,184.254 110.184,184.254 110.185,184.254 110.186,184.254 110.192,184.254 110.22,184.254 110.385,184.254 111.702,184.254 
 117.717,184.254 133.169,184.254 139.935,184.254 146.702,184.254 151.351,184.254 154.017,184.254 156.602,184.254 157.794,184.254 160.368,184.254 162.546,184.254 
 164.48

## Low Order and High Tolerance

This tests the case where accuracy is not needed as much and quick robust solutions are necessary.

In [7]:
sol = solve(prob,ode23s(),abstol=abstols[3],reltol=reltols[3],maxiters=Int(1e5))
plot(sol,ylim=[-4;4],denseplot=false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 t 
 
 
 
 
 
 y(t) 
 
 
 
 x(t)

ODE.jl fails to solve the problem and so it is removed from the tests. It should be noted that some of the higher order methods also fail to converge when the tolerances are high.

#### Final timepoint error

This measures the efficiency to get the value at the endpoint correct.

In [4]:
abstols = 1./10.^(4:7)
reltols = 1./10.^(1:4)

setups = [Dict(:alg=>Rosenbrock23()),
          Dict(:alg=>Rodas3()),
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>TRBDF2()),
          Dict(:alg=>ddebdf()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau())]
names = ["Rosenbrock23" "Rodas3" "CVODE_BDF" "TRBDF2" "ddebdf" "rodas" "radau"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;
                      names=names,save_everystep=false,appxsol=test_sol,maxiters=Int(1e5))
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 - 
 
 
 3.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2.0 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas3 
 
 
 
 CVODE 
 
 
 BDF 
 
 
 
 TRBDF2 
 
 
 
 ddebdf 
 
 
 
 rodas 
 
 
 
 radau

`CVODE_BDF` does quite well here, which is unusual for it. You can see that the low-order Rosenbrock methods `Rosenbrock23` and `Rodas3` dominate this test.

#### Timeseries error

Now we measure the average error of the timeseries.

In [5]:
abstols = 1./10.^(4:7)
reltols = 1./10.^(1:4)

setups = [Dict(:alg=>Rosenbrock23()),
          Dict(:alg=>Rodas3()),
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>TRBDF2()),
          Dict(:alg=>ddebdf()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau())]
names = ["Rosenbrock23" "Rodas3" "CVODE_BDF" "TRBDF2" "ddebdf" "rodas" "radau"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;
                      names=names,appxsol=test_sol2,maxiters=Int(1e5),errror_estimator=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas3 
 
 
 
 CVODE 
 
 
 BDF 
 
 
 
 TRBDF2 
 
 
 
 ddebdf 
 
 
 
 rodas 
 
 
 
 radau

#### Dense Errors

Now we measure the error on the interval using the interpolation.

In [6]:
abstols = 1./10.^(4:7)
reltols = 1./10.^(1:4)

setups = [Dict(:alg=>Rosenbrock23()),
          Dict(:alg=>Rodas3()),
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>TRBDF2()),
          Dict(:alg=>ddebdf()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau())]
names = ["Rosenbrock23" "Rodas3" "CVODE_BDF" "TRBDF2" "ddebdf" "rodas" "radau"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;
                      names=names,appxsol=test_sol2,maxiters=Int(1e5),errror_estimator=:L2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 4 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 - 
 
 
 3 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 Rosenbrock23 
 
 
 
 Rodas3 
 
 
 
 CVODE 
 
 
 BDF 
 
 
 
 TRBDF2 
 
 
 
 ddebdf 
 
 
 
 rodas 
 
 
 
 radau

### Higher accuracy tests

Now we transition to higher accracy tests. In this domain higher order methods are stable and much more efficient.

In [10]:
abstols = 1./10.^(7:11)
reltols = 1./10.^(4:8)
setups = [Dict(:alg=>Rodas3()),
          Dict(:alg=>GRK4A()),
          Dict(:alg=>Rodas4P()),
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>Rodas4()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau()),
          Dict(:alg=>Rodas5())]
names = ["Rodas3" "GRK4A" "Rodas4P" "CVODE_BDF" "Rodas4" "rodas" "radau" "Rodas5"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;
                      names=names,save_everystep=false,appxsol=test_sol,maxiters=Int(1e6))
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 - 
 
 
 7.5 
 
 
 10 
 
 
 - 
 
 
 5.0 
 
 
 10 
 
 
 - 
 
 
 2.5 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rodas3 
 
 
 
 GRK4A 
 
 
 
 Rodas4P 
 
 
 
 CVODE 
 
 
 BDF 
 
 
 
 Rodas4 
 
 
 
 rodas 
 
 
 
 radau 
 
 
 
 Rodas5

#### Timeseries Errors

In [11]:
abstols = 1./10.^(7:10)
reltols = 1./10.^(4:7)
setups = [Dict(:alg=>Rodas3()),
          Dict(:alg=>GRK4A()),
          Dict(:alg=>Rodas4P()),
          Dict(:alg=>CVODE_BDF()),
          Dict(:alg=>Rodas4()),
          Dict(:alg=>rodas()),
          Dict(:alg=>radau()),
          Dict(:alg=>Rodas5())]
names = ["Rodas3" "GRK4A" "Rodas4P" "CVODE_BDF" "Rodas4" "rodas" "radau" "Rodas5"]
wp = WorkPrecisionSet(prob,abstols,reltols,setups;
                      names=names,appxsol=test_sol2,maxiters=Int(1e6),error_estimate=:l2)
plot(wp)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 3 
 
 
 10 
 
 
 4 
 
 
 10 
 
 
 5 
 
 
 10 
 
 
 - 
 
 
 2 
 
 
 10 
 
 
 - 
 
 
 1 
 
 
 Error 
 
 
 Time (s) 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Rodas3 
 
 
 
 GRK4A 
 
 
 
 Rodas4P 
 
 
 
 CVODE 
 
 
 BDF 
 
 
 
 Rodas4 
 
 
 
 rodas 
 
 
 
 radau 
 
 
 
 Rodas5

The timeseries test is a little odd here because of the high peaks in the VanDerPol oscillator. At a certain accuracy, the steps try to resolve those peaks and so the error becomes higher. 

While the higher order order Julia-based Rodas methods (`Rodas4` and `Rodas4P`) Rosenbrock methods are not viable at higher tolerances, they dominate for a large portion of this benchmark. When the tolerance gets low enough, `radau` adaptive high order (up to order 13) takes the lead.

### Conclusion

`Rosenbrock23` and `Rodas3` do well when tolerances are higher. In most standard tolerances, `Rodas4` and `Rodas4P` do extremely well. Only when the tolerances get very low does `radau` do well. The Julia Rosenbrock methods vastly outperform their Fortran counterparts. `CVODE_BDF` is not a top performer in any of the tests and does not converge well.